In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import glob
import os
import seaborn as sns
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
import warnings
warnings.filterwarnings("ignore")

In [539]:
loan_df = pd.read_csv("loan_result.csv")
#log_df = pd.read_csv("log_data.csv")
user_df = pd.read_csv("user_spec.csv")

In [540]:
loan_df.columns = ["신청서_번호","한도조회_일시","금융사_번호","상품번호","승인한도","승인금리","신청_여부(타겟)"]
loan_df.head()

,신청서_번호,한도조회_일시,금융사_번호,상품번호,승인한도,승인금리,신청_여부(타겟)
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN


In [541]:
user_df.columns = ["신청서_번호","유저_번호","유저_생년월일","유저_성별","생성일시","한도조회_당시_유저_신용점수",
                  "연소득","근로형태","입사연월","고용형태","주거소유형태","대출희망금액","대출_목적",
                  "개인회생자_여부","개인회생자_납입_완료_여부","기대출수","기대출금액"]

In [137]:
# 승인한도가 없는 데이터 삭제하기
loan_df_null = loan_df[loan_df["승인한도"].isnull()]

In [138]:
user_df_drop_list = set(loan_df_null['신청서_번호'])
user_df.drop(user_df[user_df['신청서_번호'].isin(user_df_drop_list)].index,inplace=True)
user_df.sort_values(by = '생성일시',ascending = True,inplace = True)# 날짜순서로 sort
user_df.reset_index(drop = True,inplace = True)

In [139]:
user_df.isnull().sum()

신청서_번호                   0
유저_번호                    0
유저_생년월일              12903
유저_성별                12903
생성일시                     0
한도조회_당시_유저_신용점수     104459
연소득                     90
근로형태                    85
입사연월                170626
고용형태                    85
주거소유형태                  85
대출희망금액                  85
대출_목적                   85
개인회생자_여부            584678
개인회생자_납입_완료_여부     1196880
기대출수                197572
기대출금액               312331
dtype: int64

### 85개의 결측 빼기
- 대출_목적, 희망금액 ... 등 결측치가 많은 85명 우선적으로 처리하기

In [140]:
user_df = user_df[user_df["대출_목적"].notnull()]

In [141]:
user_df.isnull().sum()

신청서_번호                   0
유저_번호                    0
유저_생년월일              12901
유저_성별                12901
생성일시                     0
한도조회_당시_유저_신용점수     104456
연소득                      5
근로형태                     0
입사연월                170541
고용형태                     0
주거소유형태                   0
대출희망금액                   0
대출_목적                    0
개인회생자_여부            584593
개인회생자_납입_완료_여부     1196795
기대출수                197556
기대출금액               312309
dtype: int64

### 개인회생자 처리하기

In [142]:
user_df.loc[(user_df['개인회생자_여부']==0)&
                      (user_df['개인회생자_납입_완료_여부']==1),'개인회생자_납입_완료_여부']=0 # 개인회생자가 아니면서 납입완료된 유저는 3명 -> 완료여부 0으로 변경

user_df.loc[(user_df['개인회생자_여부']==0)&
                      (user_df['개인회생자_납입_완료_여부'].isnull()),'개인회생자_납입_완료_여부'] = 0

In [143]:
user_df['개인회생자'] = 10 #null값인 데이터는 처리 못한 상황

user_df.loc[(user_df['개인회생자_여부']==0)
                      &(user_df['개인회생자_납입_완료_여부']==0),'개인회생자'] = 0

user_df.loc[(user_df['개인회생자_여부']==1)
                      &(user_df['개인회생자_납입_완료_여부']==0),'개인회생자'] = 1

user_df.loc[(user_df['개인회생자_여부']==1)
                      &(user_df['개인회생자_납입_완료_여부']==1),'개인회생자'] = 2

In [144]:
user_df["개인회생자"].value_counts()

0     789515
10    584593
1      11321
2       1340
Name: 개인회생자, dtype: int64

### 신용점수 등급추가하기

In [145]:
bins = [0, 444, 514, 599, 664, 749, 804, 839, 869, 899, 1000]
#labels = ['null', '10등급', '9등급', '8등급', '7등급', '6등급', '5등급', '4등급', '3등급', '2등급', '1등급']
labels = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]


user_df['신용점수등급'] = pd.cut(user_df['한도조회_당시_유저_신용점수'],bins = bins, labels = labels)
user_df.loc[user_df['한도조회_당시_유저_신용점수'].isnull(),'한도조회_당시_유저_신용점수'] = 0

In [146]:
user_df["신용점수등급"].isnull().sum()

104456

### 성별 결측치 채우기

이미 기존의 데이터에 성별 및 생년월일이 존재한다면 대체

In [149]:
sex_null_list = set(user_df[(user_df['유저_번호'].isin(
    set(user_df[user_df['유저_성별'].isnull()]['유저_번호'])))&(user_df['유저_성별'].notnull())]['유저_번호'])
for i in sex_null_list:
    user_df.loc[(user_df['유저_번호']==i)&(user_df['유저_생년월일'].isnull()),
                  '유저_생년월일'] = user_df[user_df['유저_번호'] ==i]['유저_생년월일'].dropna().unique().item()
    user_df.loc[(user_df['유저_번호']==i)&(user_df['유저_성별'].isnull()),
                  '유저_성별'] = user_df[user_df['유저_번호'] ==i]['유저_성별'].dropna().unique().item()

In [150]:
user_df.isnull().sum()

신청서_번호                  0
유저_번호                   0
유저_생년월일              9680
유저_성별                9680
생성일시                    0
한도조회_당시_유저_신용점수         0
연소득                     5
근로형태                    0
입사연월               170541
고용형태                    0
주거소유형태                  0
대출희망금액                  0
대출_목적                   0
개인회생자_여부           584593
개인회생자_납입_완료_여부     584593
기대출수               197556
기대출금액              312309
개인회생자                   0
신용점수등급             104456
dtype: int64

In [177]:
train_data = user_df.dropna(how = 'any')[["유저_성별","연소득","근로형태","고용형태","주거소유형태","대출희망금액","대출_목적",
                                             "개인회생자"]]


test_data = user_df[user_df["유저_성별"].isnull()][["유저_성별","연소득","근로형태",
                                                       "고용형태","주거소유형태","대출희망금액","대출_목적","개인회생자"]]
test_data.head(3)

,유저_성별,연소득,근로형태,고용형태,주거소유형태,대출희망금액,대출_목적,개인회생자
58,NaN,24000000.0,EARNEDINCOME,정규직,전월세,30000000.0,대환대출,10
99,NaN,30000000.0,EARNEDINCOME,정규직,전월세,3000000.0,생활비,10
114,NaN,30000000.0,EARNEDINCOME,정규직,전월세,10000000.0,생활비,10


In [178]:
train_data.shape

(553869, 8)

In [179]:
data = pd.concat([train_data,test_data],axis=0)
data.shape

(563549, 8)

In [180]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

le = LabelEncoder()
for col in ["고용형태","주거소유형태","대출_목적","근로형태"]:
    data[col] = le.fit_transform(data[col])

for col in ["연소득","대출희망금액"]:
    data[col] = np.log1p(data[col])   
    
train_data = data.iloc[:553869,:]    
test_data = data.iloc[553869:,:]    
    
features_train = train_data.drop("유저_성별",axis=1)
labels_train = train_data["유저_성별"]

features_test = test_data.drop("유저_성별",axis=1)
labels_test = test_data["유저_성별"]

scaler = MinMaxScaler(feature_range = (0,1))

scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_test = scaler.transform(features_test)

X_train, X_test, y_train, y_test = train_test_split(features_train,labels_train,
                                                    random_state=0,test_size=0.2,shuffle=True,stratify=labels_train)

In [55]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {
    'C' : [0.001, 0.01, 0.1, 1, 10, 100],
    'max_iter' : [100,1000]
}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)

grid_search.fit(X_train, y_train)
print('test accuracy : ', grid_search.score(X_test, y_test))
print('최적의 parameters : ', grid_search.best_params_)

test accuracy :  0.6962555903601368
최적의 parameters :  {'C': 0.001, 'max_iter': 100}


In [181]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1_score_ = f1_score(y_test, pred)
    print('Confusion Matrix')
    print(confusion)
    print('정확도:{}, 정밀도:{}, 재현율:{}'.format(accuracy, precision, recall))
    print('f1 score:{}'.format(f1_score_))

In [182]:
lm = LogisticRegression(max_iter = 100, C=0.001)
lm.fit(X_train, y_train)
pred = lm.predict(X_test)
get_clf_eval(y_test, pred)

Confusion Matrix
[[   68 33594]
 [  121 76991]]
정확도:0.6956415765432322, 정밀도:0.6962155807749695, 재현율:0.9984308538230107
f1 score:0.8203753922545379


성별 예측진행하기

In [183]:
sex_predict = lm.predict(features_test) # 12901개의 데이터가 예측완료

In [184]:
len(sex_predict)

9680

In [185]:
user_df_sex_null = user_df[user_df["유저_성별"].isnull()].reset_index(drop=True)
user_df_sex_nonull = user_df[user_df["유저_성별"].notnull()].reset_index(drop=True)

user_df_sex_null["유저_성별"] = sex_predict

### 성별 결측치가 채워진 user_df_2

In [201]:
user_df_2 = pd.concat([user_df_sex_nonull,user_df_sex_null],axis=0)

In [202]:
user_df_2 = user_df_2.drop(["개인회생자_여부","개인회생자_납입_완료_여부"],axis=1).reset_index(drop=True)

In [203]:
user_df_2.shape

(1386769, 17)

In [204]:
user_df_2.isnull().sum()

신청서_번호                  0
유저_번호                   0
유저_생년월일              9680
유저_성별                   0
생성일시                    0
한도조회_당시_유저_신용점수         0
연소득                     5
근로형태                    0
입사연월               170541
고용형태                    0
주거소유형태                  0
대출희망금액                  0
대출_목적                   0
기대출수               197556
기대출금액              312309
개인회생자                   0
신용점수등급             104456
dtype: int64

### numeric한 데이터에 대한 이상치 처리
- 만원 단위로 낮춰서하기
- 성별, 근로형태, 고용형태, 주거소유형태 : 평균값으로 numeric 데이터 이상치 처리해주기

In [543]:
# 이상치 처리 시작
user_df_3 = user_df_2.copy()

### 소득분위 구간 추가하기
- https://insfiler.com/detail/rt_285_incomeq10-0001

In [544]:
for col in ["기대출금액","연소득","대출희망금액"]:
    user_df_3[col] = user_df_3[col] / 10000

In [545]:
bins = [-2, 70, 170, 2400, 3100, 3800, 4600, 5500, 6600, 8100, 12600,1000001]
#labels = ['null', '1분위', '9분위', '8등급', '7등급', '6등급', '5등급', '4등급', '3등급', '2등급', '1등급']
labels = [0,1,2,3,4,5,6,7,8,9,10]


user_df_3['연소득분위'] = pd.cut(user_df_3['연소득'],bins = bins, labels = labels)
user_df_3.loc[user_df_3['연소득'].isnull(),'연소득'] = 0

In [546]:
user_df_3['연소득분위'] = pd.cut(user_df_3['연소득'],bins = bins, labels = labels)

In [547]:
user_df_3.isnull().sum()

신청서_번호                  0
유저_번호                   0
유저_생년월일              9680
유저_성별                   0
생성일시                    0
한도조회_당시_유저_신용점수         0
연소득                     0
근로형태                    0
입사연월               170541
고용형태                    0
주거소유형태                  0
대출희망금액                  0
대출_목적                   0
기대출수               197556
기대출금액              312309
개인회생자                   0
신용점수등급             104456
연소득분위                   0
dtype: int64

##### 결측치 처리하기
- 성별, 근로형태, 고용형태, 주거소유형태, 연소득분위로 하나의 그룹을 생성하고 해당 그룹의 평균값으로 채워주기

- 기대출수, 기대출금액, 신용점수등급 결측치 채우기

"유저_성별","근로형태","고용형태","주거소유형태","연소득분위"

In [548]:
def get_replace_null(data,col):
    df = data.copy()

    null_data = df[df[col].isnull()]

    nonull_data = df[df[col].notnull()]
    
    # 대체할 데이터가 담긴
    standard_df = nonull_data.groupby(["유저_성별","근로형태","고용형태","주거소유형태","연소득분위"]).mean().reset_index()
    
    # null값이 대체될 데이터
    null_replace = pd.merge(null_data.drop(col,axis=1),

                  standard_df[["유저_성별","근로형태","고용형태","주거소유형태","연소득분위",col]]

             ,on=["유저_성별","근로형태","고용형태","주거소유형태","연소득분위"],how="inner")

    # 이상치 대체하고나서 concat해주기
    final = pd.concat([null_replace[df.columns],
               nonull_data],
             axis = 0)

    #final[col] = final[col].astype(int)
    
    return final.reset_index(drop=True)

"유저_성별","근로형태","고용형태","주거소유형태"

In [549]:
def get_replace_null_(data,col):
    df = data.copy()

    null_data = df[df[col].isnull()]

    nonull_data = df[df[col].notnull()]
    
    # 대체할 데이터가 담긴
    standard_df = nonull_data.groupby(["유저_성별","근로형태","고용형태","주거소유형태"]).mean().reset_index()
    
    # null값이 대체될 데이터
    null_replace = pd.merge(null_data.drop(col,axis=1),

                  standard_df[["유저_성별","근로형태","고용형태","주거소유형태",col]]

             ,on=["유저_성별","근로형태","고용형태","주거소유형태"],how="inner")

    # 이상치 대체하고나서 concat해주기
    final = pd.concat([null_replace[df.columns],
               nonull_data],
             axis = 0)

    #final[col] = final[col].astype(int)
    
    return final.reset_index(drop=True)

In [ ]:
for col in ["기대출수","기대출금액"]:
    user_df_3 = get_replace_null(user_df_3,col)
    user_df_3 = get_replace_null_(user_df_3,col)

###### 신용점수 결측치 채우기
- 신용점수가 0점인 얘들은 원래 다 결측치
- 신용점수가 0점인 얘들은 신용점수랑 신용점수 등급 날려주기

In [552]:
def make_null(data):
    d

신청서_번호                  0
유저_번호                   0
유저_생년월일              9680
유저_성별                   0
생성일시                    0
한도조회_당시_유저_신용점수         0
연소득                     0
근로형태                    0
입사연월               170540
고용형태                    0
주거소유형태                  0
대출희망금액                  0
대출_목적                   0
기대출수                    0
기대출금액                   0
개인회생자                   0
신용점수등급             104451
연소득분위                   0
dtype: int64

##### 기대출수 먼저 이상치 잡기
##### 대출희망금액 먼저 이상치 잡기

In [395]:
def find_outlier_z(data):
    
    threshold = 3
    
    mean = np.mean(data)
    std = np.std(data)
    
    z_score = [(y-mean) / std for y in data]
    
    masks = np.where(np.abs(z_score) > threshold)
    
    #print(mean+threshold*std)
    #print(mean-threshold*std)
    return masks[0]

In [396]:
def no_outlier_z(data):
    
    threshold = 3
    
    mean = np.mean(data)
    std = np.std(data)
    
    z_score = [(y-mean) / std for y in data]
    
    masks = np.where(np.abs(z_score) <= threshold)
    
    #print(mean+threshold*std)
    #print(mean-threshold*std)
    return masks[0]

결측을 채울 기준이 되는 dataframe
- 기준이 되는 dataframe : standard_df

In [377]:
def iqr_func(x):
    q3, q1 = np.percentile(x, [75, 25])
    iqr = q3 - q1
    final = q3 + 1.5*iqr
    return int(final)

In [422]:
def get_replace_outlier(data,col):
    # col에 해당하는 이상치 대체하기
    # data : 이상치 대체할 데이터프레임
    
    df = data.copy()
    
    # 이상치 대체의 기준 dataframe
    standard_df = df[df[col].notnull()].groupby(["유저_성별","근로형태","고용형태","주거소유형태","연소득분위"]
                               ).agg({col: iqr_func}).reset_index()
    
    # 이상치의 index
    outlier_index = find_outlier_z(user_df_3[col])
    
    # 이상치 아닌 것 index
    nooutlier_index = no_outlier_z(user_df_3[col])
    
    # 이상치에 속하는 데이터
    outlier_data = df.loc[outlier_index,:]
    
    # 이상치가 아닌 것에 속하는 데이터
    nooutlier_data = df.loc[nooutlier_index,:]
    
    # 이상치가 대체된 데이터프레임
    
    outlier_replace = pd.merge(outlier_data.drop(col,axis=1),
              
              standard_df[["유저_성별","근로형태","고용형태","주거소유형태","연소득분위",col]]
         
         ,on=["유저_성별","근로형태","고용형태","주거소유형태","연소득분위"],how="inner")

    
    # 이상치 대체하고나서 concat해주기
    final = pd.concat([outlier_replace[df.columns],
           nooutlier_data,
          df[df[col].isnull()]],
         axis = 0)
    
    
    return final.reset_index(drop=True)

##### user_df_4 : column의 이상치가 처리된 dataframe

In [479]:
user_df_4 = get_replace_outlier(user_df_3,"기대출수")

user_df_4 = get_replace_outlier(user_df_4,"기대출금액")

user_df_4 = get_replace_outlier(user_df_4,"대출희망금액")

user_df_4 = get_replace_outlier(user_df_4,"연소득")

##### user_df_5 : 연소득 0원 이상치가 처리된 dataframe

In [480]:
user_df_5 = user_df_4.copy()
user_df_5[user_df_5["대출희망금액"] == 0]

,신청서_번호,유저_번호,유저_생년월일,유저_성별,생성일시,한도조회_당시_유저_신용점수,연소득,근로형태,입사연월,고용형태,주거소유형태,대출희망금액,대출_목적,기대출수,기대출금액,개인회생자,신용점수등급,연소득분위
392,638976,780692,1987.0,1.0,2022-05-26 18:48:57,600.0,3450.0,EARNEDINCOME,201603.0,정규직,기타가족소유,0.0,생활비,7.0,5300.0,0,7,3
595,1255683,487083,1984.0,1.0,2022-05-12 01:29:52,580.0,5250.0,EARNEDINCOME,202008.0,정규직,자가,0.0,대환대출,11.0,8200.0,0,8,5
1019,296029,360245,1980.0,1.0,2022-04-14 15:05:32,610.0,3900.0,EARNEDINCOME,201908.0,정규직,자가,0.0,생활비,7.0,8200.0,10,7,4
1636,367209,418274,1986.0,1.0,2022-04-05 14:44:37,1000.0,12800.0,EARNEDINCOME,202105.0,정규직,전월세,0.0,생활비,1.0,NaN,10,1,9
1994,776605,676504,1991.0,0.0,2022-04-30 00:58:35,670.0,3600.0,EARNEDINCOME,202203.0,계약직,기타가족소유,0.0,대환대출,4.0,1600.0,0,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1386670,1683479,690827,NaN,0.0,2022-06-30 10:24:34,750.0,0.0,OTHERINCOME,NaN,기타,기타가족소유,0.0,생활비,NaN,NaN,0,5,0
1386685,913619,639785,NaN,0.0,2022-06-30 11:05:11,950.0,0.0,OTHERINCOME,NaN,기타,기타가족소유,0.0,생활비,NaN,NaN,0,1,0
1386697,2153455,503527,NaN,0.0,2022-06-30 11:47:58,710.0,0.0,OTHERINCOME,NaN,기타,기타가족소유,0.0,생활비,NaN,NaN,0,6,0
1386703,251831,104787,NaN,0.0,2022-06-30 12:27:22,750.0,0.0,OTHERINCOME,NaN,기타,자가,0.0,생활비,NaN,NaN,0,5,0


#### 연소득 0원 데이터 처리하기
- OTHERINCOME 기타소득을 제외하곤 연소득 0원을 평균으로 대체하기

In [481]:
user_df_5[user_df_5["연소득"] == 0]["근로형태"].value_counts()

OTHERINCOME        17245
FREELANCER          1352
PRIVATEBUSINESS      750
EARNEDINCOME         525
EARNEDINCOME2        203
PRACTITIONER          22
Name: 근로형태, dtype: int64

- 근로형태에 따라서 하나씩 평균값 대입

In [482]:
df_lst = []

for work_type in ['FREELANCER', 'PRIVATEBUSINESS', 'EARNEDINCOME', 'EARNEDINCOME2',
       'PRACTITIONER']:
    
    # 연소득 0원 채워야하는 데이터
    replaced = user_df_5[(user_df_5["연소득"] == 0) & (user_df_5["근로형태"] == work_type)]

    # 연소득 0원이 아닌 데이터
    no_replace = user_df_5[(user_df_5["연소득"] != 0) & (user_df_5["근로형태"] == work_type)]
    
    standard_df = user_df_5.groupby(["유저_성별","근로형태","고용형태","주거소유형태"]).mean().reset_index()
   
    # 0값이 처리된 데이터
    zero_replace = pd.merge(replaced.drop("연소득",axis=1),

                  standard_df[["유저_성별","근로형태","고용형태","주거소유형태","연소득"]]

             ,on=["유저_성별","근로형태","고용형태","주거소유형태"],how="inner")

    zero_replace = zero_replace[user_df_5.columns] # 0값이 처리된 데이터
    zero_replace["연소득"] = zero_replace["연소득"].astype(int)
    
    print(work_type,"전처리 끝")
    
    df_lst.append(zero_replace)
    df_lst.append(no_replace)
    
df_lst.append(user_df_5[user_df_5["근로형태"] == "OTHERINCOME"])

FREELANCER 전처리 끝
PRIVATEBUSINESS 전처리 끝
EARNEDINCOME 전처리 끝
EARNEDINCOME2 전처리 끝
PRACTITIONER 전처리 끝


#### 대출희망금액 0원 전처리
- "유저_성별","근로형태","고용형태","주거소유형태","연소득분위" 기반 평균으로 대체
- 그래도 0이면?
- "유저_성별","근로형태","고용형태","주거소유형태" 기반 평균으로 대체

In [483]:
user_df_5 = pd.concat(df_lst,axis=0).reset_index(drop=True)

In [484]:
user_df_5.shape

(1386769, 18)

In [485]:
user_df_5[user_df_5["대출희망금액"] == 0]["근로형태"].value_counts()

EARNEDINCOME       2250
OTHERINCOME        1807
FREELANCER          397
EARNEDINCOME2       260
PRIVATEBUSINESS     114
PRACTITIONER         73
Name: 근로형태, dtype: int64

In [486]:
# 대출희망금액 0원 채워야하는 데이터
replaced = user_df_5[(user_df_5["대출희망금액"] == 0)]

# 대출희망금액 0원이 아닌 데이터
no_replace = user_df_5[(user_df_5["대출희망금액"] != 0)]
    
standard_df = user_df_5.groupby(["유저_성별","근로형태","고용형태","주거소유형태","연소득분위"]).mean().reset_index()

In [487]:
# 0값이 처리된 데이터
zero_replace = pd.merge(replaced.drop("대출희망금액",axis=1),
                  standard_df[["유저_성별","근로형태","고용형태","주거소유형태","연소득분위","대출희망금액"]]

             ,on=["유저_성별","근로형태","고용형태","주거소유형태","연소득분위"],how="inner")

zero_replace = zero_replace[user_df_5.columns] # 0값이 처리된 데이터
zero_replace["대출희망금액"] = zero_replace["대출희망금액"].astype(int)

In [488]:
user_df_5 = pd.concat([zero_replace,no_replace],axis=0).reset_index(drop=True)

"유저_성별","근로형태","고용형태","주거소유형태" 기반 평균으로 대체

In [489]:
# 대출희망금액 0원 채워야하는 데이터
replaced = user_df_5[(user_df_5["대출희망금액"] == 0)]

# 대출희망금액 0원이 아닌 데이터
no_replace = user_df_5[(user_df_5["대출희망금액"] != 0)]
    
standard_df = user_df_5.groupby(["유저_성별","근로형태","고용형태","주거소유형태"]).mean().reset_index()

# 0값이 처리된 데이터
zero_replace = pd.merge(replaced.drop("대출희망금액",axis=1),
                  standard_df[["유저_성별","근로형태","고용형태","주거소유형태","대출희망금액"]]

             ,on=["유저_성별","근로형태","고용형태","주거소유형태"],how="inner")

zero_replace = zero_replace[user_df_5.columns] # 0값이 처리된 데이터
zero_replace["대출희망금액"] = zero_replace["대출희망금액"].astype(int)

user_df_5 = pd.concat([zero_replace,no_replace],axis=0).reset_index(drop=True)

############### 이상치 처리 끝 #####################

In [490]:
user_df_5

,신청서_번호,유저_번호,유저_생년월일,유저_성별,생성일시,한도조회_당시_유저_신용점수,연소득,근로형태,입사연월,고용형태,주거소유형태,대출희망금액,대출_목적,기대출수,기대출금액,개인회생자,신용점수등급,연소득분위
0,19242,855132,1982.0,0.0,2022-04-22 14:57:23,860.0,3249.0,EARNEDINCOME,202010.0,일용직,자가,3593.0,생활비,1.0,NaN,1,3,0
1,19242,855132,1982.0,0.0,2022-04-22 14:57:23,860.0,3249.0,EARNEDINCOME,202010.0,일용직,자가,3593.0,생활비,1.0,NaN,1,3,0
2,1451516,274024,1973.0,1.0,2022-06-25 19:16:30,680.0,3827.0,PRACTITIONER,20020725.0,기타,전월세,2464.0,생활비,3.0,2300.0,0,6,0
3,1061944,647131,2003.0,0.0,2022-04-23 11:47:38,0.0,2903.0,PRACTITIONER,202201.0,계약직,기타가족소유,2214.0,생활비,NaN,NaN,0,NaN,0
4,1061944,647131,2003.0,0.0,2022-04-23 11:47:38,0.0,2903.0,PRACTITIONER,202201.0,계약직,기타가족소유,2214.0,생활비,NaN,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1386764,415760,562410,NaN,1.0,2022-06-30 17:37:42,470.0,3000.0,OTHERINCOME,NaN,기타,자가,300.0,생활비,NaN,NaN,0,9,3
1386765,1629388,92748,NaN,1.0,2022-06-30 17:46:34,250.0,2000.0,OTHERINCOME,NaN,기타,자가,100.0,생활비,NaN,NaN,0,10,2
1386766,957433,773358,NaN,1.0,2022-06-30 17:54:05,470.0,3000.0,OTHERINCOME,NaN,기타,자가,300.0,생활비,NaN,NaN,0,9,3
1386767,1131790,821338,NaN,1.0,2022-06-30 18:50:37,470.0,3000.0,OTHERINCOME,NaN,기타,자가,300.0,생활비,NaN,NaN,0,9,3
